<a href="https://colab.research.google.com/github/AbrahamKong/CMPE257-Regression_MLP_Latent_Manifolds/blob/main/CMPE257_Regression_MLP_Latent_Manifolds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Statement

Author: Abraham Kong

Team: Data Miners

Objective: 
Using Regression methods to find out if Delivery is one parameter effecting the churn rate of customers

### Data Set

[Brazilian E-Commerce Public Dataset by Olist](https://www.kaggle.com/olistbr/brazilian-ecommerce)


In [1]:
#Import Necessary Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import plotly.express as px
from sklearn.mixture import GaussianMixture
from sklearn.cluster import Birch
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster.elbow import kelbow_visualizer
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the Necessary Datasets

customers = pd.read_csv('/content/olist_customers_dataset.csv')
order_items = pd.read_csv('/content/olist_order_items_dataset.csv')
order_payments= pd.read_csv('/content/olist_order_payments_dataset.csv')
orders = pd.read_csv('/content/olist_orders_dataset.csv')
sellers= pd.read_csv('/content/olist_sellers_dataset.csv')
products = pd.read_csv('/content/olist_products_dataset.csv')
geolocations = pd.read_csv('/content/olist_geolocation_dataset.csv')

In [3]:
geolocations.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [4]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
# Merge all the Data into one dataframe

df1 = order_payments.merge(order_items, on="order_id")
df2 = df1.merge(orders, on="order_id")
df3 = df2.merge(customers, on="customer_id")
df4 = df3.merge(products, on="product_id")
df = df4.merge(sellers, on="seller_id")
df

,order_id,payment_sequential,payment_type,payment_installments,payment_value,order_item_id,product_id,seller_id,shipping_limit_date,price,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33,1,af74cc53dcffc8384b29e7abfa41902b,213b25e6f54661939f11710a6fddb871,2018-05-02 22:15:09,79.80,...,57.0,921.0,8.0,800.0,17.0,27.0,17.0,13321,salto,SP
1,d86305c284cecff56c7ee9d3fbd4f8d6,1,credit_card,4,93.72,1,af74cc53dcffc8384b29e7abfa41902b,213b25e6f54661939f11710a6fddb871,2018-04-19 23:10:02,79.80,...,57.0,921.0,8.0,800.0,17.0,27.0,17.0,13321,salto,SP
2,5308fdab519f7478ef64894128fbbee4,1,credit_card,1,36.07,1,1b93af229a5768eaaed2064722b1c1dc,213b25e6f54661939f11710a6fddb871,2018-06-11 12:11:27,129.95,...,57.0,915.0,5.0,9950.0,52.0,22.0,52.0,13321,salto,SP
3,5308fdab519f7478ef64894128fbbee4,2,voucher,1,112.67,1,1b93af229a5768eaaed2064722b1c1dc,213b25e6f54661939f11710a6fddb871,2018-06-11 12:11:27,129.95,...,57.0,915.0,5.0,9950.0,52.0,22.0,52.0,13321,salto,SP
4,bb54e9bf4bdb6c6391a7c6b517748b9f,1,credit_card,2,124.92,1,71540d636652fc682032d3b3c0a180c1,213b25e6f54661939f11710a6fddb871,2018-06-04 11:50:16,113.85,...,54.0,923.0,8.0,1050.0,16.0,27.0,16.0,13321,salto,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117596,ef8c51f3610d990552701e064e71291f,1,credit_card,1,2028.75,1,e245d0b2b8c02f14e840abef856cc149,8ec76bb0965af3f007692b26fa9d6623,2017-11-23 13:30:38,1999.00,...,56.0,1654.0,4.0,525.0,19.0,10.0,17.0,88804,criciuma,SC
117597,76a8885167de6cebce9e33f8abc9f149,1,boleto,1,72.52,1,3954b1f7435552939dd7fdc5cbfea1e4,c7246702dc991a06d788120dc7544391,2018-02-28 03:31:25,54.86,...,35.0,106.0,2.0,1300.0,25.0,10.0,26.0,60440,fortaleza,CE
117598,24f6d178b92691453bffa42151277fbf,1,boleto,1,78.12,1,4b6c0fa189ae4c4c7a4d357ff185512c,52831e5990d0771cb4951d6f12928167,2018-08-13 04:35:20,65.00,...,36.0,959.0,1.0,250.0,25.0,10.0,18.0,17600,tupa,SP
117599,1b88975e652b474825e1d68839f836be,1,credit_card,1,51.54,1,ee13af1ea8a18c0c1862a20778da9eb4,e116c7455dd26a4d8c3e92532583905f,2018-04-11 11:08:11,18.38,...,36.0,1000.0,1.0,525.0,18.0,12.0,12.0,1140,sao paulo,SP


In [6]:
# Transfer String Date to Datetime and remove time section

df['order_approved_at'] = pd.to_datetime(df['order_approved_at'], format='%Y-%m-%d').dt.normalize()
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'], format='%Y-%m-%d').dt.normalize()
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'], format='%Y-%m-%d').dt.normalize()
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'], format='%Y-%m-%d').dt.normalize()

In [7]:
# Caluculate Delivery Time

df['delivery_time'] = df['order_delivered_customer_date'] - df['order_approved_at']
df['estimate_delivery_time'] = df['order_estimated_delivery_date'] - df['order_approved_at']

In [8]:
# Calculate the Estimate time Error rate
# (Actual Delivery Time - Estimated Delivery Time) / Estimated Delivery Time

df['estimate_delivery_time_difference'] = (
    pd.to_numeric(df['delivery_time'].dt.days, downcast='float') -
    pd.to_numeric(df['estimate_delivery_time'].dt.days, downcast='float')
)/pd.to_numeric(df['estimate_delivery_time'].dt.days, downcast='float')

df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value,order_item_id,product_id,seller_id,shipping_limit_date,price,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,delivery_time,estimate_delivery_time,estimate_delivery_time_difference
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33,1,af74cc53dcffc8384b29e7abfa41902b,213b25e6f54661939f11710a6fddb871,2018-05-02 22:15:09,79.80,...,800.0,17.0,27.0,17.0,13321,salto,SP,14 days,27 days,-0.481481
1,d86305c284cecff56c7ee9d3fbd4f8d6,1,credit_card,4,93.72,1,af74cc53dcffc8384b29e7abfa41902b,213b25e6f54661939f11710a6fddb871,2018-04-19 23:10:02,79.80,...,800.0,17.0,27.0,17.0,13321,salto,SP,5 days,24 days,-0.791667
2,5308fdab519f7478ef64894128fbbee4,1,credit_card,1,36.07,1,1b93af229a5768eaaed2064722b1c1dc,213b25e6f54661939f11710a6fddb871,2018-06-11 12:11:27,129.95,...,9950.0,52.0,22.0,52.0,13321,salto,SP,18 days,41 days,-0.560976
3,5308fdab519f7478ef64894128fbbee4,2,voucher,1,112.67,1,1b93af229a5768eaaed2064722b1c1dc,213b25e6f54661939f11710a6fddb871,2018-06-11 12:11:27,129.95,...,9950.0,52.0,22.0,52.0,13321,salto,SP,18 days,41 days,-0.560976
4,bb54e9bf4bdb6c6391a7c6b517748b9f,1,credit_card,2,124.92,1,71540d636652fc682032d3b3c0a180c1,213b25e6f54661939f11710a6fddb871,2018-06-04 11:50:16,113.85,...,1050.0,16.0,27.0,16.0,13321,salto,SP,30 days,38 days,-0.210526


In [9]:
df.columns

Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value',
       'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'delivery_time', 'estimate_delivery_time',
       'estimate_delivery_time_difference'],
      dtype='object')

In [10]:
# Calculate Distances between Sellers and Customers

# Find out Longtitude and Latitude for Sellers

Functions

In [11]:
# Extracts the geolocation coordinates 
# from the sellers/costumers zip code digits 
# using the olist_geolocation_dataset.csv file



def zip_to_geo(data, zip_code):
  zip_rows = data.loc[data['geolocation_zip_code_prefix'] == zip_code].reset_index() # Might have multiple geolocation for each zip code
  lat = zip_rows['geolocation_lat'].mean() # Check this approach 
  lon = zip_rows['geolocation_lng'].mean()

  return lat, lon

In [12]:
def distance_between_customer_and_seller(customer_zip, seller_zip):
  return haversine_distance(*zip_to_geo(geolocations, customer_zip),*zip_to_geo(geolocations, seller_zip))

In [13]:
# Calculate the great circle distance in kilometers between two points 
# on the earth (specified in decimal degrees)

from math import radians, cos, sin, asin, sqrt


##


def haversine_distance(lat1, lon1, lat2, lon2):
  r = 6371 # Planet earth radius in kilometres
  phi1 = np.radians(lat1)
  phi2 = np.radians(lat2)
  delta_phi = np.radians(lat2 - lat1)
  delta_lambda = np.radians(lon2 - lon1)
  a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
  res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))

  return np.round(res, 2)

####

def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return np.round(c * r, 2)

Continuous code

In [14]:
# Calculating distance between customer and seller


from tqdm.notebook import tqdm

# Import to fix DataFrame.progress_apply
tqdm.pandas()

# Calculate the haversine distance between Sellers and Customers using the longtitude and latitude
df['distance_customer_seller'] = df.progress_apply(lambda row : distance_between_customer_and_seller(row['customer_zip_code_prefix'], row['seller_zip_code_prefix']), axis = 1)


  0%|          | 0/117601 [00:00<?, ?it/s]

In [15]:
# Creation of Training dataset

df_training = df[[
 'customer_city',
 'customer_state',
 'seller_city',
 'seller_state',
 'product_category_name',  
 'distance_customer_seller',
 'estimate_delivery_time',
 'estimate_delivery_time_difference'
]]
df_training.head()

,customer_city,customer_state,seller_city,seller_state,product_category_name,distance_customer_seller,estimate_delivery_time,estimate_delivery_time_difference
0,teofilo otoni,MG,salto,SP,casa_construcao,845.89,27 days,-0.481481
1,barretos,SP,salto,SP,casa_construcao,322.74,24 days,-0.791667
2,goiania,GO,salto,SP,casa_construcao,748.96,41 days,-0.560976
3,goiania,GO,salto,SP,casa_construcao,748.96,41 days,-0.560976
4,matias barbosa,MG,salto,SP,casa_construcao,433.48,38 days,-0.210526


In [16]:
# Converting estimate_delivery_time from days to float

df_training['estimate_delivery_time'] = pd.to_numeric(df_training['estimate_delivery_time'].dt.days, downcast='float')

In [17]:
# Removing delayed delivery as they can be outliers

df.drop(df_training[df_training.estimate_delivery_time_difference> 0].index, inplace=True)

In [18]:
# Converting categorical values into numeric

from sklearn import preprocessing

def encoding_categoricals(df=None):
  encoders = []
  col_names = df.columns
  for col in col_names:
    if type(df[col].iloc[0]) == str:
      le = preprocessing.LabelEncoder()
      le.fit(df[col].astype(str))
      encoders.append(le)
      df.loc[:,col] = le.transform(df[col].astype(str))
  return df, encoders


In [19]:
df_encoded, encoders = encoding_categoricals(df_training)
df_encoded.head()

,customer_city,customer_state,seller_city,seller_state,product_category_name,distance_customer_seller,estimate_delivery_time,estimate_delivery_time_difference
0,3830,10,466,22,16,845.89,27.0,-0.481481
1,420,25,466,22,16,322.74,24.0,-0.791667
2,1442,8,466,22,16,748.96,41.0,-0.560976
3,1442,8,466,22,16,748.96,41.0,-0.560976
4,2263,10,466,22,16,433.48,38.0,-0.210526


In [20]:
# Removes NaN, infinite and negative infinite values from the dataset

In [21]:
def clean_dataset(df):
  assert isinstance(df, pd.DataFrame), "df is not pd.DataFrame"

  df.dropna(inplace=True)
  indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
  return df[indices_to_keep].astype(np.float32)

In [22]:
df_encoded_clean = clean_dataset(df_encoded)

In [23]:
# Normalizing the dataset

def normalize_dataset(df=None):
  column_names = df.columns

  scaler = preprocessing.StandardScaler()
  X = scaler.fit_transform(df[column_names[:-1]])

  new_df = pd.DataFrame(data=X, columns=column_names[:-1])
  df = pd.concat([new_df, df['estimate_delivery_time_difference']], axis=1)

  return df, scaler

In [24]:
df_training, scaler = normalize_dataset(df_encoded_clean)
df_training.head()

,customer_city,customer_state,seller_city,seller_state,product_category_name,distance_customer_seller,estimate_delivery_time,estimate_delivery_time_difference
0,1.202995,-1.235993,0.644534,0.527029,-0.949565,0.423236,0.345607,-0.481481
1,-1.623026,0.891261,0.644534,0.527029,-0.949565,-0.465860,0.006286,-0.791667
2,-0.776049,-1.519627,0.644534,0.527029,-0.949565,0.258503,1.929109,-0.560976
3,-0.776049,-1.519627,0.644534,0.527029,-0.949565,0.258503,1.929109,-0.560976
4,-0.095649,-1.235993,0.644534,0.527029,-0.949565,-0.277657,1.589787,-0.210526


In [25]:
# Imputing for missing data

from sklearn.impute import SimpleImputer

def fill_missing_data(df=None):
  imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
  df.iloc[:,:] = imputer.fit_transform(df)

  return df

In [26]:
df_training_full = fill_missing_data(df_training)

### Split and Train

In [27]:
df_training_full.head()

,customer_city,customer_state,seller_city,seller_state,product_category_name,distance_customer_seller,estimate_delivery_time,estimate_delivery_time_difference
0,1.202995,-1.235993,0.644534,0.527029,-0.949565,0.423236,0.345607,-0.481481
1,-1.623026,0.891261,0.644534,0.527029,-0.949565,-0.465860,0.006286,-0.791667
2,-0.776049,-1.519627,0.644534,0.527029,-0.949565,0.258503,1.929109,-0.560976
3,-0.776049,-1.519627,0.644534,0.527029,-0.949565,0.258503,1.929109,-0.560976
4,-0.095649,-1.235993,0.644534,0.527029,-0.949565,-0.277657,1.589787,-0.210526


In [28]:
X = df_training_full.drop(columns = ['estimate_delivery_time_difference'])
X.columns

Index(['customer_city', 'customer_state', 'seller_city', 'seller_state',
       'product_category_name', 'distance_customer_seller',
       'estimate_delivery_time'],
      dtype='object')

In [29]:
y = df_training_full['estimate_delivery_time_difference']
y

0        -0.481481
1        -0.791667
2        -0.560976
3        -0.560976
4        -0.210526
            ...   
117594   -0.666667
117595   -0.315789
117596   -0.565217
117597   -0.173913
117598   -0.714286
Name: estimate_delivery_time_difference, Length: 117326, dtype: float32

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Correlation between features and target

In [31]:
focus_cols = ['estimate_delivery_time_difference']
df_encoded_clean.corr().filter(focus_cols).drop(focus_cols).style.format('{:.2}')

,estimate_delivery_time_difference
customer_city,-0.017
customer_state,-0.12
seller_city,-0.00054
seller_state,0.021
product_category_name,-0.0047
distance_customer_seller,0.14
estimate_delivery_time,-0.058


#### What to classify?

Customer
Seller reliability
Churn - Low , medium and high

## Modeling

### Linear Regression

In [32]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)
lr_result = lr.predict(X_test)

In [33]:
print(lr_result)

[-0.5048984  -0.48177198 -0.51354855 ... -0.48293555 -0.47463945
 -0.5026693 ]


In [34]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score

print("mean_sqrd_error is=",mean_squared_error(y_test,lr_result))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,lr_result)))

mean_sqrd_error is= 0.15811214
root_mean_squared error of is= 0.39763317


### DecisionTreeRegressor



In [35]:
from sklearn.tree import DecisionTreeRegressor

regr = DecisionTreeRegressor(max_depth=2)
regr.fit(X_train, y_train)
regr_result = regr.predict(X_test)

In [36]:
print(regr_result)

[-0.51740865 -0.4838514  -0.51740865 ... -0.4838514  -0.4838514
 -0.50043931]


In [37]:
regr.score(X_test, y_test)

0.0005113440008278625

In [38]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,regr_result)
print("r2 score is",score)
print("mean_sqrd_error is=",mean_squared_error(y_test,regr_result))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,regr_result)))


r2 score is 0.0005113440008278625
mean_sqrd_error is= 0.15810288269325787
root_mean_squared error of is= 0.39762153197891315


### MLPRegressor



In [39]:
from sklearn.neural_network import MLPRegressor

regr_MLP = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
regr_result = regr_MLP.predict(X_test)

print(regr_result)

[-0.47691458 -0.41363716 -0.48250788 ... -0.45435137 -0.44775718
 -0.5283435 ]


In [40]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score

print("mean_sqrd_error is=",mean_squared_error(y_test,regr_result))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,regr_result)))

mean_sqrd_error is= 0.15879764
root_mean_squared error of is= 0.3984942


### RandomForestRegressor



In [41]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, y_train)
rf_result = rf.predict(X_test)


In [42]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score

print("mean_sqrd_error is=",mean_squared_error(y_test,rf_result))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,rf_result)))

mean_sqrd_error is= 0.17132336815800606
root_mean_squared error of is= 0.4139122710889423


### KNNRegressor

In [43]:
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train, y_train)

neigh_result = neigh.predict(X_test)

In [44]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score

print("mean_sqrd_error is=",mean_squared_error(y_test,neigh_result))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,neigh_result)))

mean_sqrd_error is= 0.21882474
root_mean_squared error of is= 0.46778706


### XBoost Regressor

In [45]:
import xgboost
from sklearn import ensemble

params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

xgb = ensemble.GradientBoostingRegressor(**params)
xgb.fit(X_train, y_train)

xgb_result = xgb.predict(X_test)

In [46]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score

print("mean_sqrd_error is=",mean_squared_error(y_test,xgb_result))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,xgb_result)))

mean_sqrd_error is= 0.15826996560852516
root_mean_squared error of is= 0.3978315794510601
